# Cover Type Prediction using ensembles

## Dataset Description
The dataset represents the data about trees which were planted in the US. The dataset consists of the information about 500000 trees. Your aim is to build  Random Forest Ensemble to predict the cover type of trees. In order to successfully complete this assignment you have to follow this algorithm:
* Load the training data
* Transform categorical features into vector representations
* Split dataset into the train and validation part
* Fit the Random Forest Ensemble into the training set
* Compare the accuracy of the fitted model with the Logistic Regression Model, which is about 0.67 for this set


If you have enough time, it will be very interesting to dig into further research through these steps:
* Determine which features are valuable for your model (calculate feature importance of your model).
* Try to reduce number of trees and see the results.
* Understand why the linear models have poor performance on this dataset.


## Loading data

Init pyspark session

In [1]:
from __future__ import division, print_function, unicode_literals # For the compatibility with Python 2

In [2]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder\
                            .enableHiveSupport()\
                            .appName("sparkml")\
                            .master("local[4]")\
                            .getOrCreate()

Load train dataset located at /data/covertype2 with at least 60 partitions (use function repartition for this case). Use option `inferSchema` to save numerical features.

In [3]:
df = spark_session.read.format("com.databricks.spark.csv")\
        .option("header", "true")\
        .option("inferschema", "true")\
        .option("mode", "DROPMALFORMED")\
        .load("/data/covertype2/train.csv")\
        .repartition(60)

## Transforming data

As you can see, there are two categorical features in dataset: 'Soil_Type' and 'Wild_Type'. You have to transform them into the vector embeddings.

First of all, you have to use StringIndexer to transform feature types to indexes

In [4]:
from pyspark.ml.feature import StringIndexer

In [5]:
cat_cols=['Soil_Type','Wild_Type']
cat_cols_index={'Soil_Type':'Soil_Index','Wild_Type':'Wild_Index'}
cat_cols_encoder={'Soil_Index':'SoilEncoder','Wild_Index':'WildEncoder'}


In [6]:
cat_indexers = [ StringIndexer(inputCol=col, outputCol=cat_cols_index[col])
                 for col in cat_cols ]

Apply OneHotEncoder technique to the dataset in order to get vectors for the Random Forest classification

In [7]:
from pyspark.ml.feature import OneHotEncoder

In [8]:
encoders = [ OneHotEncoder(dropLast=False, inputCol=indexer.getOutputCol(),
                           outputCol=cat_cols_encoder[indexer.getOutputCol()])
             for indexer in cat_indexers ]


Use the VectorAssembler technique to accumulate all features into one vector. Don't forget to use features that you have generated

In [9]:
from pyspark.ml.feature import VectorAssembler

In [10]:
vector_assembler = VectorAssembler(inputCols=['SoilEncoder', # feature name of Soil type encoded
                                              'WildEncoder', # feature name of Wild type encoded
                                              'Elevation',
                                              'Aspect',
                                              'Slope',
                                              'Horizontal_Distance_To_Hydrology',
                                              'Vertical_Distance_To_Hydrology',
                                              'Horizontal_Distance_To_Roadways',
                                              'Hillshade_9am',
                                              'Hillshade_Noon',
                                              'Hillshade_3pm',
                                              'Horizontal_Distance_To_Fire_Points'
                                              ], outputCol='features')

In [11]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier

## Training

Fit the Random Forest model to the train dataset. Don't forget to split dataset into two parts to check your trained models. It is desirable to use about 100 trees with depth about 7 in order to avoid wasting too much time waiting while your model will be fit to the data. Try to adjust the options 'subsamplingRate' and 'featureSubsetStrategy' to get better results

<b> Extra task.</b> Use the Cross-Validation to check your model.

In [12]:
#Pipeline for static model
rf = RandomForestClassifier(labelCol='Target',featuresCol='features', numTrees=100,maxDepth=9)
pipeline = Pipeline(stages=[ *cat_indexers, *encoders, vector_assembler, rf ] )

In [13]:
#Training the model
(trainingData, testData) = df.randomSplit([0.8, 0.2], seed = 123)
model = pipeline.fit(trainingData)

Get the feature importances of the trained model. What 5 features are the most important in the dataset?

#model.stages[5].featureImportances

Apply model to the validation part of your set and get the accuracy score for the data. Use the MulticlassClassificationEvaluator class from the ml.evaluation module. 

In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [15]:
predictions = model.transform(testData)
evaluator = MulticlassClassificationEvaluator(labelCol = "Target", predictionCol = "prediction", metricName = "accuracy")
accuracy = evaluator.evaluate(predictions)
print(accuracy) 

0.7339568438559734


**Adding CV **

In [19]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
rfcv = RandomForestClassifier(labelCol='Target',featuresCol='features')
pipelinecv = Pipeline(stages=[ *cat_indexers, *encoders, vector_assembler, rfcv ] )
paramGrid = ParamGridBuilder()\
    .addGrid(rf.maxDepth, [7,8,10])\
    .addGrid(rf.numTrees,[80,90,100])\
    .build()

In [20]:
evaluator = MulticlassClassificationEvaluator(labelCol = "Target", predictionCol = "prediction", metricName = "accuracy") 
crossval = CrossValidator(estimator = pipelinecv,
                          estimatorParamMaps = paramGrid,
                          evaluator = evaluator,
                          numFolds = 10)

In [21]:
cvModel = crossval.fit(df)
print(cvModel.avgMetrics)
print(cvModel.bestModel)


[0.6723369649735333, 0.6723369649735333, 0.6723369649735333, 0.6723369649735333, 0.6723369649735333, 0.6723369649735333, 0.6723369649735333, 0.6723369649735333, 0.6723369649735333]
PipelineModel_4baca3fef45b734e4ef3


Are your results better than the results from the Logistic Regression model?

# Performing test submission

Apply the models to the test dataset.

<b>Note!</b> Dataset will be changed during the test phase. Your last cell output must be the accuracy score.

In [22]:
# Load dataset, transform dataset
dfTest = spark_session.read.format("com.databricks.spark.csv")\
        .option("header", "true")\
        .option("inferschema", "true")\
        .option("mode", "DROPMALFORMED")\
        .load("/data/covertype2/test.csv")\
        .repartition(60)


In [23]:
# Calculate accuracy for static model
dfTestfinal=model.transform(dfTest)
evaluator = MulticlassClassificationEvaluator(labelCol = "Target", predictionCol = "prediction", metricName = "accuracy")
accuracy = evaluator.evaluate(dfTestfinal)
print(round(accuracy,2))


0.73


In [24]:
#Calculate accuracy for cv best model
dfTestfinalcv=cvModel.bestModel.transform(dfTest)
evaluator = MulticlassClassificationEvaluator(labelCol = "Target", predictionCol = "prediction", metricName = "accuracy")
accuracy = evaluator.evaluate(dfTestfinalcv)
print(round(accuracy,2))
spark_session.stop()

0.66
